In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib as plt

In [2]:
data=pd.read_csv("Score.csv")
#data

In [3]:
col=np.array(data.columns)
col=np.delete(col,range(3))
#col

In [4]:
'''
maxScore1=[100 for i in range(36)]
maxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]
maxScore=maxScore1+maxScore2
'''

'\nmaxScore1=[100 for i in range(36)]\nmaxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]\nmaxScore=maxScore1+maxScore2\n'

In [5]:
transDataDict={}
for i in range(55):
    temp=(data[col[i]]-np.mean(data[col[i]]))/(np.max(data[col[i]])-np.min(data[col[i]]))
    transDataDict[col[i]]=temp
transData=pd.DataFrame.from_dict(transDataDict)
del transDataDict

In [6]:
mergeData=transData.apply(lambda x:[x['CHI611'],x['CHI612'],x['CHI621'],x['CHI622'],
x['CHI711'],x['CHI712'],x['CHI721'],x['CHI722'],
x['CHI811'],x['CHI812'],x['CHI821'],x['CHI822'],
x['CHI911'],x['CHI912'],x['CHI921']],axis=1)
mergeData=pd.DataFrame(mergeData,columns=['CHI'])
mergeData['CHINoNan']=mergeData['CHI'].apply(lambda x:[i for i in x if not np.isnan(i)])
mergeData['CHILen']=mergeData['CHINoNan'].apply(lambda x:len(x))

In [7]:
def merge(x,col):
    if col in ['MATH','ENG']:
        x[col]=transData.apply(lambda x:[x[col+'611'],x[col+'612'],x[col+'621'],x[col+'622'],
        x[col+'711'],x[col+'712'],x[col+'721'],x[col+'722'],
        x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'PHY':
        x[col]=transData.apply(lambda x:[x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'CHE':
        x[col]=transData.apply(lambda x:[x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
def NoNan(x,col):
    x[col+'NoNan']=x[col].apply(lambda x:[i for i in x if not np.isnan(i)])
    x[col+'Len']=x[col+'NoNan'].apply(lambda x:len(x))
def delOrigin(x,col):
    x.drop([col],axis=1,inplace=True)

delOrigin(mergeData,'CHI')

for i in ['MATH','ENG','PHY','CHE']:
    merge(mergeData,i)
    NoNan(mergeData,i)
    delOrigin(mergeData,i)

In [8]:
def normTest(x,col):
    if x[col+'Len']>=3:
        return stats.shapiro(x[col+'NoNan'])
    else:
        return (-1,-1)

for i in ['CHI','MATH','ENG']:
    mergeData[i+'Norm']=mergeData.apply(lambda x:normTest(x,i),axis=1)

In [9]:
def stdTest(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    return stats.levene(temp[0],temp[1],temp[2])
mergeData['stdTest']=mergeData.apply(stdTest,axis=1)

In [10]:
equalStdData=mergeData[mergeData['stdTest'].apply(lambda x:x[1]>0.05)].copy()

In [11]:
def oneWay(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    return stats.f_oneway(temp[0],temp[1],temp[2])
equalStdData['oneWay']=equalStdData.apply(oneWay,axis=1)

In [12]:
len(equalStdData[equalStdData['oneWay'].apply(lambda x:x[1]>0.05)])

54

In [13]:
equalStdData['isBias']=equalStdData['oneWay'].apply(lambda x:x[1]<0.05)

In [14]:
sexBiasCount=equalStdData.join(data['Sex'])
sexBiasCount=sexBiasCount.groupby(['Sex','isBias'])['CHINoNan'].count().reset_index()
sexBiasCount.rename(columns={'CHINoNan':'count'},inplace=True)
sexBiasCount=sexBiasCount.pivot(index='Sex',columns='isBias',values='count')

In [15]:
sexBiasCount

isBias,False,True
Sex,,
F,23,64
M,25,71


In [16]:
stats.chi2_contingency(sexBiasCount)

(0.011571704382183943,
 0.9143353224117214,
 1,
 array([[22.81967213, 64.18032787],
        [25.18032787, 70.81967213]]))

In [17]:
for i in ['CHI','MATH','ENG']:
    mergeData[i+'Mean']=mergeData[i+'NoNan'].apply(np.mean)

In [18]:
def findBias(df):
    meanCol=['CHIMean','MATHMean','ENGMean']
    dis=np.zeros(3)
    maxScoreLesson=np.argmax(df[meanCol])
    for i in range(3):
        temp=0
        for j in meanCol:
            temp+=np.abs(df[meanCol[i]]-df[j])
        dis[i]=temp
    argMax=np.argmax(dis)
    flag=(maxScoreLesson==argMax)
    return (np.max(dis),argMax,flag)
mergeData['bias']=mergeData.apply(findBias,axis=1)

In [19]:
mergeData['biasDis']=mergeData['bias'].apply(lambda x:x[0])
mergeData['biasClass']=mergeData['bias'].apply(lambda x:x[1])
mergeData['biasPostive']=mergeData['bias'].apply(lambda x:x[2])

In [20]:
biasData=equalStdData.join(mergeData[['biasDis','biasClass','biasPostive']])[['isBias','biasDis','biasClass','biasPostive']]
biasData=biasData.join(data['Sex'])

In [21]:
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','biasPostive'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='biasPostive',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)

biasClass     0   1   2
biasPostive            
False        40  19  18
True         19  23  24


(7.913342173027404,
 0.01912667949269471,
 2,
 array([[31.76923077, 22.61538462, 22.61538462],
        [27.23076923, 19.38461538, 19.38461538]]))

In [22]:
#已经偏科的人，偏科的科目和偏科是优势还是劣势不独立
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','biasPostive'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='biasPostive',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)

biasClass     0   1   2
biasPostive            
False        40  19  18
True         19  23  24


(7.913342173027404,
 0.01912667949269471,
 2,
 array([[31.76923077, 22.61538462, 22.61538462],
        [27.23076923, 19.38461538, 19.38461538]]))

In [23]:
classPostiveData=biasData[biasData['biasPostive']>0].groupby(['isBias','biasClass'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='isBias',columns='biasClass',values='biasDis')
stats.chi2_contingency(temp)

(1.1825275261282586,
 0.5536271888574527,
 2,
 array([[ 8.63829787, 10.12765957,  9.23404255],
        [20.36170213, 23.87234043, 21.76595745]]))

In [24]:
classPostiveData=biasData[.groupby(['isBias','biasClass'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='isBias',columns='biasClass',values='biasDis')
print(temp)
print(stats.chi2_contingency(temp))

SyntaxError: invalid syntax (<ipython-input-24-cb28c9079fde>, line 1)

In [25]:
# 已经偏科的人，偏科科目和性别无关
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)
classPostiveData=biasData[(biasData['isBias']>0)&(biasData['biasPostive']>0)].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)
classPostiveData=biasData[(biasData['isBias']>0)&(biasData['biasPostive']==0)].groupby(['biasClass','Sex'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='Sex',columns='biasClass',values='biasDis')
print(temp)
stats.chi2_contingency(temp)

biasClass   0   1   2
Sex                  
F          23  17  24
M          33  20  18
biasClass   0   1   2
Sex                  
F          11   8  14
M           7  14  10
biasClass   0  1   2
Sex                 
F          12  9  10
M          26  6   8


(4.918300320694208,
 0.08550758783687076,
 2,
 array([[16.5915493 ,  6.54929577,  7.85915493],
        [21.4084507 ,  8.45070423, 10.14084507]]))

In [26]:
noEqualStdData=mergeData[mergeData['stdTest'].apply(lambda x:x[1]<0.05)].copy()

In [27]:
'''
for i in ['CHI','MATH','ENG']:
    noEqualStdData[i+'Std']=mergeData[i+'NoNan'].apply(np.std)
noEqualStdData[['CHIMean','MATHMean','ENGMean','CHIStd','MATHStd','ENGStd']]
'''

"\nfor i in ['CHI','MATH','ENG']:\n    noEqualStdData[i+'Std']=mergeData[i+'NoNan'].apply(np.std)\nnoEqualStdData[['CHIMean','MATHMean','ENGMean','CHIStd','MATHStd','ENGStd']]\n"

In [28]:
def kruskal(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    return stats.kruskal(temp[0],temp[1],temp[2])
noEqualStdData['kruskal']=noEqualStdData.apply(kruskal,axis=1)
noEqualStdData['isBias']=noEqualStdData['kruskal'].apply(lambda x:x[1]<0.05)

In [44]:
isBias=pd.concat([noEqualStdData['isBias'],equalStdData['isBias']],axis=0,ignore_index=True)
data.merge()

0      False
1       True
2       True
3      False
4       True
       ...  
237     True
238     True
239    False
240     True
241     True
Name: isBias, Length: 242, dtype: bool

In [51]:
mergeData[(mergeData['CHILen']<8) | (mergeData['MATHLen']<8) | (mergeData['ENGLen']<8)]

,CHINoNan,CHILen,MATHNoNan,MATHLen,ENGNoNan,ENGLen,PHYNoNan,PHYLen,CHENoNan,CHELen,...,MATHNorm,ENGNorm,stdTest,CHIMean,MATHMean,ENGMean,bias,biasDis,biasClass,biasPostive
11,"[0.12465734649122817, 0.03987927048548682, 0.0...",4,"[0.11052631578947368, 0.1342288011695906, 0.10...",4,"[0.1267056530214424, 0.21802131912058625, 0.17...",4,[],0,[],0,...,"(0.8320170640945435, 0.17312289774417877)","(0.8823816180229187, 0.3489076793193817)","(3.665009667123008, 0.06849378201937291)",0.091798,0.113233,0.160064,"(0.11509750027795942, 2, True)",0.115098,2,True
35,"[-0.18784265350877183, -0.10717955304392494, -...",4,"[-0.08947368421052632, -0.08452119883040939, -...",4,"[-0.0955165692007798, -0.2503331112591606, -0....",4,[],0,[],0,...,"(0.7149626016616821, 0.017109088599681854)","(0.9721591472625732, 0.8548051118850708)","(1.056338024466466, 0.3871750370414262)",-0.121809,-0.118744,-0.239675,"(0.2387973272717478, 2, False)",0.238797,2,False
75,"[-0.03159265350877183, -0.033650141279219066, ...",7,"[-0.04736842105263158, -0.0011878654970760572,...",7,"[0.003248862897985622, -0.03514323784143905, -...",7,[],0,[],0,...,"(0.9532236456871033, 0.7589210867881775)","(0.9593520164489746, 0.8131561875343323)","(0.05141067481167504, 0.9500274547364366)",0.023393,-0.005724,0.005820,"(0.04669004098615128, 0, True)",0.046690,0,True
76,"[0.20278234649122817, 0.12811456460313386, 0.0...",4,"[0.11052631578947368, 0.1967288011695906, 0.16...",4,"[0.10201429499675105, 0.1294137241838774, 0.14...",4,[],0,[],0,...,"(0.95697021484375, 0.7598424553871155)","(0.8830317854881287, 0.35174646973609924)","(0.6172761212897977, 0.5607657496780332)",0.128687,0.156483,0.132146,"(0.05213259432986078, 1, True)",0.052133,1,True
77,"[0.11180849511020295, 0.07196724843783675, 0.2...",3,"[0.15082956259426855, 0.09606060606060601, 0.1...",3,"[0.19926905673512005, 0.1935583597987691, 0.08...",3,[],0,[],0,...,"(0.853529691696167, 0.24993479251861572)","(0.7873522043228149, 0.08464788645505905)","(0.26629566378844305, 0.7748136820868523)",0.130852,0.116896,0.159244,"(0.07073997615668774, 2, True)",0.070740,2,True
82,"[0.22539113428943933, 0.18864468864468867, 0.1...",7,"[0.1891356066613798, 0.26957070707070707, 0.21...",7,"[0.19841628959276014, 0.19738854895104893, 0.2...",7,"[0.23720810439560427, 0.1592242503259453, 0.28...",7,"[0.24336538461538465, 0.14726928016635074, 0.1...",3,...,"(0.9884002804756165, 0.9898721575737)","(0.7362639904022217, 0.009097964502871037)","(1.0431677782409738, 0.37268861276630866)",0.173929,0.209188,0.205300,"(0.06662963488437487, 0, False)",0.066630,0,False
106,[0.09870279989725152],1,[],0,[],0,[],0,[],0,...,"(-1, -1)","(-1, -1)","(nan, nan)",0.098703,NaN,NaN,"(nan, 0, True)",NaN,0,True
125,"[0.15590734649122817, 0.02517338813254564, -0....",7,"[0.04736842105263158, 0.10297880116959061, -0....",7,"[0.052631578947368335, -0.06045969353764158, 0...",7,[],0,[],0,...,"(0.9050085544586182, 0.3623952269554138)","(0.83884197473526, 0.0969165787100792)","(1.5331376172118225, 0.24275196819281272)",-0.009775,0.049283,0.017791,"(0.09054912157218456, 1, True)",0.090549,1,True
129,"[0.09340734649122817, 0.03987927048548682, 0.0...",7,"[0.06842105263157895, 0.11339546783625727, 0.1...",7,"[0.13905133203378808, 0.06612258494337107, 0.0...",7,[],0,[],0,...,"(0.9363880753517151, 0.6064329147338867)","(0.9699529409408569, 0.8981099128723145)","(0.7336606132707865, 0.493965695653951)",0.032353,0.066436,0.030954,"(0.0695644684628805, 1, True)",0.069564,1,True
150,"[-0.028846153846153903, -0.03357753357753354, ...",7,"[0.04480570975416329, 0.05744949494949495, 0.0...",7,"[-0.09570135746606337, -0.029884178321678334, ...",7,"[-0.35207760989011, -0.23060625814863095, -0.0...",7,"[0.05105769230769233, 0.03883554522659171, -0....",3,...,"(0.8689687252044678, 0.18180660903453827)","(0.8163676857948303, 0.059236958622932434)","(1.5892317397572528, 0.23142086391836075)",-0.061981,0.026107,-0.050495,"(0.16469050475806385, 1, True)",0.164691,1,True
